In [1]:
import tensorflow as tf 

In [2]:
(train_x, train_y), (test_x, test_y) = tf.keras.datasets.mnist.load_data()
train_x = train_x / .255 
test_x = test_x / .255

In [3]:
train_x = train_x[..., tf.newaxis] 
test_x = test_x[..., tf.newaxis]

In [4]:
train_ds = tf.data.Dataset.from_tensor_slices((train_x, train_y))
train_ds = train_ds.shuffle(100) 
train_ds = train_ds.batch(32) 

test_ds = tf.data.Dataset.from_tensor_slices((test_x, test_y))
# test_ds = test_ds.shuffle(100) 
test_ds = test_ds.batch(32) 

In [5]:
train_x.shape, test_x.shape

((60000, 28, 28, 1), (10000, 28, 28, 1))

In [6]:
inputs = tf.keras.layers.Input((28, 28, 1))

net = tf.keras.layers.Conv2D(32, (3, 3), padding='same')(inputs)
net = tf.keras.layers.Activation('relu')(net)
net = tf.keras.layers.Conv2D(32, (3, 3), padding='same')(net)
net = tf.keras.layers.Activation('relu')(net)
net = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(net)
net = tf.keras.layers.Dropout(0.25)(net)

net = tf.keras.layers.Conv2D(32, (3, 3), padding='same')(inputs)
net = tf.keras.layers.Activation('relu')(net)
net = tf.keras.layers.Conv2D(32, (3, 3), padding='same')(net)
net = tf.keras.layers.Activation('relu')(net)
net = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(net)
net = tf.keras.layers.Dropout(0.25)(net)

net = tf.keras.layers.Flatten()(net)
net = tf.keras.layers.Dense(512)(net)
net = tf.keras.layers.Activation('relu')(net)
net = tf.keras.layers.Dropout(0.5)(net)
net = tf.keras.layers.Dense(10)(net)
net = tf.keras.layers.Activation('softmax')(net)

model = tf.keras.Model(inputs = inputs, outputs = net)


In [7]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy() 
optimizer = tf.keras.optimizers.Adam() 

In [8]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [11]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)

@tf.function
def test_step(images, labels):
    # with tf.GradientTape() as tape:
    predictions = model(images)
    loss = loss_object(labels, predictions)
    # gradients = tape.gradient(loss, model.trainable_variables)
    # optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    test_loss(loss)
    test_accuracy(labels, predictions)

In [10]:
for epoch in range(10): 

    for images, labels in train_ds:
        train_step(images, labels)

    for images, labels in test_ds:
        test_step(images, labels)

    temp = 'Epoch : {} . Loss : {} . Accuracy : {} . Test Loss : {} . Test Accuracy : {}'

    print(temp.format(epoch + 1
                    , train_loss.result()
                    , train_accuracy.result() * 100
                    , test_loss.result()
                    , test_accuracy.result() * 100))


Epoch : 1 . Loss : 1.104750394821167 . Accuracy : 94.68333435058594 . Test Loss : 0.14416088163852692 . Test Accuracy : 95.87000274658203
Epoch : 2 . Loss : 0.5853633880615234 . Accuracy : 96.3800048828125 . Test Loss : 0.1273391842842102 . Test Accuracy : 96.6500015258789
Epoch : 3 . Loss : 0.40934374928474426 . Accuracy : 97.0361099243164 . Test Loss : 0.1263166218996048 . Test Accuracy : 96.80332946777344
Epoch : 4 . Loss : 0.3200131952762604 . Accuracy : 97.42666625976562 . Test Loss : 0.12227043509483337 . Test Accuracy : 97.00250244140625
Epoch : 5 . Loss : 0.26427149772644043 . Accuracy : 97.71299743652344 . Test Loss : 0.12615768611431122 . Test Accuracy : 97.07599639892578
Epoch : 6 . Loss : 0.22637809813022614 . Accuracy : 97.94361114501953 . Test Loss : 0.12545011937618256 . Test Accuracy : 97.21833801269531
Epoch : 7 . Loss : 0.19870273768901825 . Accuracy : 98.1145248413086 . Test Loss : 0.12678784132003784 . Test Accuracy : 97.32428741455078
Epoch : 8 . Loss : 0.177729979